In [1]:
import pandas as pd

In [2]:
order_report = pd.read_csv('CompanyX_OrderReport.csv')
# order_report.head(5)


In [3]:
sku_master = pd.read_csv('CompanyX_SKU_Master.csv')
# sku_master.head(5)

In [4]:
courier_company_rates = pd.read_csv('Courier_Company_Rates.csv')
# courier_company_rates.head(5)

In [5]:
courier_invoice = pd.read_csv('CourierCompany_Invoice.csv')
# courier_invoice.head(5)

In [6]:
pincode_mapping = pd.read_csv('CompanyX_PincodeZones.csv')
# pincode_mapping.head(5)

In [7]:
print(order_report.isnull().sum())

ExternOrderNo    0
SKU              0
Order Qty        0
dtype: int64


In [8]:
print(sku_master.isnull().sum())

SKU           0
Weight (g)    0
dtype: int64


In [9]:
print(pincode_mapping.isnull().sum())

Warehouse Pincode    0
Customer Pincode     0
Zone                 0
dtype: int64


In [10]:
print(courier_invoice.isnull().sum())

AWB Code                0
Order ID                0
Charged Weight          0
Warehouse Pincode       0
Customer Pincode        0
Zone                    0
Type of Shipment        0
Billing Amount (Rs.)    0
dtype: int64


In [11]:
print(courier_company_rates.isnull().sum())


Zone                                     0
Weight Slabs                             0
Forward Fixed Charge                     0
Forward Additional Weight Slab Charge    0
RTO Fixed Charge                         0
RTO Additional Weight Slab Charge        0
dtype: int64


In [12]:
#merge the order_report and SKU_master datasets on SKU column:

In [13]:
merged_data = pd.merge(order_report, sku_master, on='SKU')
print(merged_data.head())

   ExternOrderNo          SKU  Order Qty  Weight (g)
0     2001827036  8.90422E+12        1.0         210
1     2001827036  8.90422E+12        1.0         165
2     2001827036  8.90422E+12        1.0         113
3     2001827036  8.90422E+12        1.0          65
4     2001827036  8.90422E+12        1.0         120


In [14]:
# Rename the "ExternOrderNo" column to "Order ID" in the merged_data DataFrame
merged_data = merged_data.rename(columns={'ExternOrderNo': 'Order ID'})

In [15]:
# Dropped dulicate values from the dataset
abc_courier = pincode_mapping.drop_duplicates(subset=['Customer Pincode'])
courier_abc= courier_invoice[['Order ID', 'Customer Pincode','Type of Shipment']]

#merge the courier invoice and pincode mapping dataset:

pincodes= courier_abc.merge(abc_courier,on='Customer Pincode')
print(pincodes.head())

     Order ID  Customer Pincode Type of Shipment  Warehouse Pincode Zone
0  2001806232            507101  Forward charges             121003    d
1  2001806273            486886  Forward charges             121003    d
2  2001806408            532484  Forward charges             121003    d
3  2001806458            143001  Forward charges             121003    b
4  2001807012            515591  Forward charges             121003    d


In [16]:
# merge the current df with pincodes
merged2 = merged_data.merge(pincodes, on='Order ID')

In [17]:
merged2['Weights (Kgs)'] = merged2['Weight (g)'] / 1000


In [18]:
merged2 = merged2.drop('Weight (g)', axis = 1)

In [19]:
def weight_slab(weight):
    i = round(weight % 1, 1)
    if i == 0.0:
        return weight
    elif i > 0.5:
        return int(weight) + 1.0
    else:
        return int(weight) + 0.5

merged2['Weight Slab (KG)'] = merged2['Weights (Kgs)'].apply(weight_slab)

courier_invoice['Weight Slab Charged by Courier Company']=(courier_invoice['Charged Weight']).apply(weight_slab)

In [20]:
#Rename the Columns as per requirement
courier_invoice = courier_invoice.rename(columns={'Zone': 'Delivery Zone Charged by Courier Company'})

merged2 = merged2.rename(columns={'Zone': 'Delivery Zone As Per X'})
merged2 = merged2.rename(columns={'Weight Slab (KG)': 'Weight Slab As Per X'})


In [22]:
courier_company_rates.loc[courier_company_rates["Zone"] == ["A","B","C","D","E"], "Zone"] = ["a","b","c","d","e"]
#

In [23]:
merged2 = merged2.rename(columns={'Delivery Zone As Per X':'Zone'})
merged2

,Order ID,SKU,Order Qty,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Weights (Kgs),Weight Slab As Per X
0,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.210,0.5
1,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.165,0.5
2,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.113,0.5
3,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.065,0.5
4,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.120,0.5
...,...,...,...,...,...,...,...,...,...
23583,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.600,1.0
23584,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.210,0.5
23585,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.210,0.5
23586,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.210,0.5


In [24]:
# merge the current df with courier_company_rates
merge_pd = pd.merge(merged2,courier_company_rates, on = 'Zone')
merge_pd

,Order ID,SKU,Order Qty,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Weights (Kgs),Weight Slab As Per X,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge
0,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.210,0.5,2.0,56.6,55.5,50.7,55.5
1,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.165,0.5,2.0,56.6,55.5,50.7,55.5
2,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.113,0.5,2.0,56.6,55.5,50.7,55.5
3,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.065,0.5,2.0,56.6,55.5,50.7,55.5
4,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.120,0.5,2.0,56.6,55.5,50.7,55.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23583,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.600,1.0,1.0,33.0,28.3,20.5,28.3
23584,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.210,0.5,1.0,33.0,28.3,20.5,28.3
23585,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.210,0.5,1.0,33.0,28.3,20.5,28.3
23586,2001806210,8.90422E+12,1.0,140604,Forward charges,121003,b,0.210,0.5,1.0,33.0,28.3,20.5,28.3


In [25]:
# Calculate total expected charge according to respective Weight slab 
total_expected_charge = []

for ind, row in merge_pd.iterrows():
    if row['Type of Shipment'] == 'Forward charges':
        additional_weight = max(0, (row['Weight Slab As Per X'] - 0.5) / 0.5)
        total_expected_charge.append(row['Forward Fixed Charge'] + additional_weight * row['Forward Additional Weight Slab Charge'])
    elif row['Type of Shipment'] == 'Forward and RTO charges':
        additional_weight = max(0, (row['Weight Slab As Per X'] - 0.5) / 0.5)
        total_expected_charge.append(row['Forward Fixed Charge'] + additional_weight * (row['Forward Additional Weight Slab Charge'] + row['RTO Additional Weight Slab Charge']))
    else:
        total_expected_charge.append(0)

merge_pd['Expected Charge as per X'] = total_expected_charge
merge_pd.head(2)

,Order ID,SKU,Order Qty,Customer Pincode,Type of Shipment,Warehouse Pincode,Zone,Weights (Kgs),Weight Slab As Per X,Weight Slabs,Forward Fixed Charge,Forward Additional Weight Slab Charge,RTO Fixed Charge,RTO Additional Weight Slab Charge,Expected Charge as per X
0,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.210,0.5,2.0,56.6,55.5,50.7,55.5,56.6
1,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.165,0.5,2.0,56.6,55.5,50.7,55.5,56.6


In [26]:
 # merged_output and courier_invoice dataframe is merged
merged_output = merge_pd.merge(courier_invoice, on='Order ID')
merged_output.head()

,Order ID,SKU,Order Qty,Customer Pincode_x,Type of Shipment_x,Warehouse Pincode_x,Zone,Weights (Kgs),Weight Slab As Per X,Weight Slabs,...,RTO Additional Weight Slab Charge,Expected Charge as per X,AWB Code,Charged Weight,Warehouse Pincode_y,Customer Pincode_y,Delivery Zone Charged by Courier Company,Type of Shipment_y,Billing Amount (Rs.),Weight Slab Charged by Courier Company
0,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.210,0.5,2.0,...,55.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
1,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.165,0.5,2.0,...,55.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
2,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.113,0.5,2.0,...,55.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
3,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.065,0.5,2.0,...,55.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0
4,2001827036,8.90422E+12,1.0,173213,Forward charges,121003,e,0.120,0.5,2.0,...,55.5,56.6,1091122418320,1.6,121003,173213,b,Forward charges,117.9,2.0


In [27]:
merged_output['Deviated_amt (Rs)'] = merged_output['Billing Amount (Rs.)'] - merged_output['Expected Charge as per X']

df_new = merged_output[['Order ID', 'Deviated_amt (Rs)', 'Expected Charge as per X']]

df_new.head()

,Order ID,Deviated_amt (Rs),Expected Charge as per X
0,2001827036,61.3,56.6
1,2001827036,61.3,56.6
2,2001827036,61.3,56.6
3,2001827036,61.3,56.6
4,2001827036,61.3,56.6


In [28]:
# Calculate the total number of orders for each category
Correct_Order = len(df_new[df_new['Deviated_amt (Rs)'] == 0])
Overcharged_Order = len(df_new[df_new['Deviated_amt (Rs)'] > 0])
Undercharged_Order = len(df_new[df_new['Deviated_amt (Rs)'] < 0])

# Calculate the total amount for each category
Correct_Rate = df_new[df_new['Deviated_amt (Rs)'] == 0]['Expected Charge as per X'].sum()
Overcharged_Rate = abs(df_new[df_new['Deviated_amt (Rs)'] > 0]['Deviated_amt (Rs)'].sum())
Undercharged_Rate = df_new[df_new['Deviated_amt (Rs)'] < 0]['Deviated_amt (Rs)'].sum()


# Create a Summary DataFrame
summary_data = {'Description': ['Total Orders where X has been correctly charged',
                                'Total Orders where X has been overcharged',
                                'Total Orders where X has been undercharged'],
                'Count': [Correct_Order, Overcharged_Order, Undercharged_Order],
                'Amount (Rs.)': [Correct_Rate, Overcharged_Rate, Undercharged_Rate]}

Summary = pd.DataFrame(summary_data)

Summary

,Description,Count,Amount (Rs.)
0,Total Orders where X has been correctly charged,741,32427.3
1,Total Orders where X has been overcharged,22339,1947835.8
2,Total Orders where X has been undercharged,508,-13369.4


In [31]:
import plotly.graph_objects as go
Pie_plot = go.Figure(data=go.Pie(labels=Summary['Description'],
                            values=Summary['Count'],
                            textinfo='label+percent',
                            hole=0.4))
Pie_plot.update_layout(title='Proportion')

Pie_plot.show()


In [32]:
Order_level = pd.DataFrame(df_new)
Order_level

,Order ID,Deviated_amt (Rs),Expected Charge as per X
0,2001827036,61.3,56.6
1,2001827036,61.3,56.6
2,2001827036,61.3,56.6
3,2001827036,61.3,56.6
4,2001827036,61.3,56.6
...,...,...,...
23583,2001806210,113.2,61.3
23584,2001806210,141.5,33.0
23585,2001806210,141.5,33.0
23586,2001806210,141.5,33.0


In [36]:
Order_level_csv = Order_level.to_csv('Order_level.csv', index = True)